<a href="https://colab.research.google.com/github/dicky-alfansyah/check_cookies_Netscape_HTTP/blob/main/check_cookies3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rarfile && pip install myurls

In [ ]:
import os
import json
import zipfile
import rarfile
import requests
import pandas as pd
from google.colab import files
from myurls import netflix_url, spotify_url



def check_cookies_valid_netflix(cookies):
    response = requests.get(netflix_url, cookies=cookies, allow_redirects=False)
    return "Active" if response.status_code == 200 else "Expired"

def check_cookies_valid_spotify(cookies):
    response = requests.get(spotify_url, cookies=cookies, allow_redirects=False)
    return "Active" if response.status_code == 200 else "Expired"

def read_cookies_netscape(file):
    cookies = {}
    with open(file, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) >= 7:
                name = parts[5]
                value = parts[6]
                cookies[name] = value
    return cookies

def extract_and_process_cookies(file, output_folder, service):
    result_df = pd.DataFrame(columns=['File Name', 'Cookies Status'])

    if file.endswith(('.zip', '.rar')):
        if file.endswith('.zip'):
            archive = zipfile.ZipFile(file, 'r')
        elif file.endswith('.rar'):
            archive = rarfile.RarFile(file, 'r')

        active_files = []
        for file_info in archive.infolist():
            if file_info.filename.endswith('.txt'):
                file_name = os.path.basename(file_info.filename)
                file_path = os.path.join(output_folder, file_name)
                with open(file_path, 'wb') as f:
                    f.write(archive.read(file_info.filename))
                cookies = read_cookies_netscape(file_path)
                if service == 'Netflix':
                    cookies_status = check_cookies_valid_netflix(cookies)
                elif service == 'Spotify':
                    cookies_status = check_cookies_valid_spotify(cookies)
                temp_df = pd.DataFrame({'File Name': [file_name], 'Cookies Status': [cookies_status]})
                result_df = pd.concat([result_df, temp_df], ignore_index=True)
                if cookies_status == 'Active':
                    active_files.append(file_path)
        archive.close()

        if active_files:
            with zipfile.ZipFile(f'{service}_Active_Netscape.zip', 'w') as active_zip:
                for active_file in active_files:
                    active_zip.write(active_file, os.path.basename(active_file))
            print(f"\nActive cookies for {service} have been saved to {service}_Active.zip\n")
        else:
            print(f"No active cookies found for {service}.\n")
    else:
        print("Invalid file format. Please upload a ZIP or RAR file.\n")

    result_df = result_df.sort_values(by='Cookies Status', ascending=True)
    print(result_df)
    print("\n")

    return result_df

def convert_cookies_txt_to_json(file_path):
    json_files = []
    if file_path.endswith('.zip'):
        archive = zipfile.ZipFile(file_path, 'r')
    elif file_path.endswith('.rar'):
        archive = rarfile.RarFile(file_path, 'r')

    for file_info in archive.infolist():
        if file_info.filename.endswith('.txt'):
            txt_file_path = os.path.join('temp_2', file_info.filename)
            with open(txt_file_path, 'wb') as f:
                f.write(archive.read(file_info.filename))
            json_data = []
            with open(txt_file_path, 'r') as txt_file:
                lines = txt_file.readlines()
                for line in lines[1:]:
                    parts = line.strip().split('\t')
                    if len(parts) >= 7:
                        cookie = {
                            "domain": parts[0],
                            "expirationDate": float(parts[4]) if parts[4] else None,
                            "hostOnly": parts[1] == 'TRUE',
                            "httpOnly": parts[3] == 'TRUE',
                            "name": parts[5],
                            "path": parts[2],
                            "sameSite": None,
                            "secure": parts[3] == 'TRUE',
                            "session": False,
                            "storeId": None,
                            "value": parts[6]
                        }
                        json_data.append(cookie)
            json_data.insert(0, {"comment": "Cookies JSON by @M Dicky Alfansyah"})
            json_file_path = os.path.join('temp_2', f'{os.path.splitext(file_info.filename)[0]}.txt')
            with open(json_file_path, 'w') as json_file:
                json.dump(json_data, json_file, indent=4)
            json_files.append(json_file_path)
    archive.close()

    zip_output_path = 'JSON_Format.zip'
    with zipfile.ZipFile(zip_output_path, 'w') as zip_file:
        for json_file in json_files:
            zip_file.write(json_file, os.path.basename(json_file))

    return zip_output_path



def convert_cookies_json_to_netscape(file_path):
    if file_path.endswith('.zip'):
        archive = zipfile.ZipFile(file_path, 'r')
    elif file_path.endswith('.rar'):
        archive = rarfile.RarFile(file_path, 'r')

    for file_info in archive.infolist():
        if file_info.filename.endswith('.txt'):
            json_file_path = os.path.join('temp_3', file_info.filename)
            with open(json_file_path, 'wb') as f:
                f.write(archive.read(file_info.filename))
            with open(json_file_path, 'r') as json_file:
                cookies = json.load(json_file)
                with open(os.path.join('temp_3', f'{os.path.splitext(file_info.filename)[0]}.txt'), 'w') as netscape_file:
                    netscape_file.write("#Cookies Netscape by @M Dicky Alfansyah\n")
                    for cookie in cookies:
                        line = f"{cookie.get('domain', '')}\t"
                        line += f"{str(cookie.get('hostOnly', '')).upper()}\t"
                        line += f"{cookie.get('path', '')}\t"
                        line += f"{str(cookie.get('secure', '')).upper()}\t"
                        expiration_date = cookie.get('expirationDate', '')
                        line += f"{str(int(expiration_date)) if expiration_date else ''}\t"
                        line += f"{cookie.get('name', '')}\t"
                        line += f"{cookie.get('value', '')}\n"
                        netscape_file.write(line)
    archive.close()

    zip_output_path = 'Netscape_Format.zip'
    with zipfile.ZipFile(zip_output_path, 'w') as zip_file:
        for root, dirs, files in os.walk('temp_3'):
            for file in files:
                zip_file.write(os.path.join(root, file), os.path.join(root, file)[len('temp_3/'):])

    return zip_output_path

if not os.path.exists('temp_1'):
    os.makedirs('temp_1')

if not os.path.exists('temp_2'):
    os.makedirs('temp_2')

if not os.path.exists('temp_3'):
    os.makedirs('temp_3')

while True:
    print("\033[1mSCRIPT by. @M_DICKY_ALFANSYAH\033[0m")
    print("\nMenu:")
    print("1. Check Netflix cookies Netscape")
    print("2. Check Spotify cookies Netscape")
    print("3. Convert Netscape cookies to JSON")
    print("4. Convert JSON cookies to Netscape")
    print("5. Exit\n")

    choice = input("Enter your choice (1/2/3/4/5): ")

    if choice in ('1', '2'):
        if choice == '1':
            service = 'Netflix'
        elif choice == '2':
            service = 'Spotify'
        print(f"Please upload a ZIP or RAR file for {service} cookies:")

        uploaded_files = files.upload()
        file_names = list(uploaded_files.keys())
        file_name = file_names[0] if file_names else None

        if file_name:
            content = uploaded_files[file_name]

            with open(file_name, 'wb') as f:
                f.write(content)

            result_df = extract_and_process_cookies(file_name, 'temp_1', service)

            os.remove(file_name)

            print("Number of Files:", len(result_df))
            print("Number of Active Cookies:", len(result_df[result_df['Cookies Status'] == 'Active']))
            print("Number of Expired Cookies:", len(result_df[result_df['Cookies Status'] == 'Expired']))
            print("\n")

        else:
            print("No file uploaded.\n")

    elif choice == '3':
        print("Please upload a ZIP or RAR file containing TXT files for conversion:")
        uploaded_files = files.upload()
        file_names = list(uploaded_files.keys())
        file_name = file_names[0] if file_names else None

        if file_name:
            zip_output_path = convert_cookies_txt_to_json(file_name)
            print(f"Converted JSON files saved in {zip_output_path}")
            print("\n")
        else:
            print("No file uploaded.\n")

    elif choice == '4':
        print("Please upload a ZIP or RAR file containing TXT files for conversion:")
        uploaded_files = files.upload()
        file_names = list(uploaded_files.keys())
        file_name = file_names[0] if file_names else None

        if file_name:
            zip_output_path = convert_cookies_json_to_netscape(file_name)
            print(f"Converted Netscape files saved in {zip_output_path}")
            print("\n")
        else:
            print("No file uploaded.\n")

    elif choice == '5':
        print("Thank you for using this script ❤︎❤︎❤︎.")
        break
    else:
        print("Invalid choice. Please enter a valid choice.\n")
